In [90]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import seaborn as sns
import math
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import *
import sklearn.model_selection
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.svm import *
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.feature_extraction.text import *

import emot

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\carol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\carol\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Client support

# 1. Data extraction

In [12]:
filename = "../../data/extraction_twint/tweets_data/data_service_clients/with_reply_time/amazon.csv"

In [14]:
full_df = pd.read_csv(filename)
df = full_df[["tweet","reply_time"]].copy()
df= df.rename(columns={'tweet':'raw'})

In [15]:
full_df.columns

Index(['id', 'conversation_id', 'created_at', 'timezone', 'user_id',
       'username', 'name', 'place', 'tweet', 'mentions', 'urls', 'photos',
       'replies_count', 'retweets_count', 'likes_count', 'hashtags',
       'cashtags', 'link', 'retweet', 'quote_url', 'video', 'near', 'geo',
       'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to',
       'retweet_date', 'translate', 'trans_src', 'trans_dest', 'datetime',
       'reply_time'],
      dtype='object')

In [16]:
df.columns

Index(['raw', 'reply_time'], dtype='object')

In [17]:
df.head()

raw  \
0  Wt a disgusting amazon ! @JeffBezos @AmitAgarw...   
1  "On the way but running late" is what I always...   
2  7866061659 is call for refund your money this ...   
3                         It’s from Amazon @AmazonUK   
4                   Check DM. \nI have sent details.   

                  reply_time  
0                        NaN  
1  0 days 00:02:42.000000000  
2                        NaN  
3                        NaN  
4                        NaN

In [18]:
sum(df.reply_time.isnull())

1657

In [19]:
df.shape

(6839, 2)

In [20]:
#reply time to timedelta
df['reply_time']=df['reply_time'].map(pd.to_timedelta)

#reply time in minutes
df['reply_time']=df['reply_time'].map(lambda x: x.days*1440 + x.seconds//60)

In [26]:
df.dtypes

raw            object
reply_time    float64
dtype: object

In [25]:
df["raw"] = df["raw"].astype('str')

# 2. Pre-processing

In [28]:
df['has_reply'] = df['reply_time'].map(lambda x : not(math.isnan(x)))

In [29]:
df['has_reply'].value_counts()

True     5182
False    1657
Name: has_reply, dtype: int64

### Change text

In [75]:
def remove_uppercase(text):
    return text.lower()
    
def remove_punctuation(text):
    """custom function to remove the punctuation"""  
    PUNCT_TO_REMOVE = string.punctuation
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    STOPWORDS = set(stopwords.words('english'))
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def remove_frequent_words(text):
    print(text)
    FREQWORDS = get_frequent_words(df)
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

# def remove_urls(text):
#     url_pattern = re.compile(r'https?://\S+|www\.\S+')
#     expression = re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*')
#     text =  url_pattern.sub(r'', text)
#     return expression.sub(r'', text)

def remove_urls(text):
    words=[]
    for w in text.split(" "):
        if not(w.startswith("http") or w.startswith("www") or w.startswith("pic.twitter")):
            words+=[w]
    return ' '.join(words)


In [32]:
def get_frequent_words(column):
    cnt = Counter()
    for text in column.values:
        for word in text.split():
            cnt[word] += 1
    FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
    #RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-10-1:-1]])
    return FREQWORDS

get_frequent_words(df.raw)

{'@AmazonHelp', 'I', 'a', 'and', 'is', 'it', 'my', 'the', 'to', 'you'}

### Emoticons to words

In [44]:
def convert_emojis(text):
    #for emot in UNICODE_EMO:
     #   text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
    emo= emot.emoji(text)
    #print(emo)
    for i in range(len(emo['value'])):
        text = text.replace(emo['value'][i], emo['mean'][i]).replace(":","")
    return text

#text = "game is on 🔥 🔥 😭"
#print(convert_emojis(text))

### Chat words conversion

In [55]:
words = "../../source/slang.txt"
with open(words, 'r') as file:
    chat_words_str = file.read()

In [56]:
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

#chat_words_conversion("one minute BRB")

In [59]:
#df.raw.apply(chat_words_conversion)

### Stemming

In [60]:
#not so good results

def stem_words(text):
    stemmer = PorterStemmer()
    return " ".join([stemmer.stem(word) for word in text.split()])

#df["stemmed"] = df["raw"].apply(lambda text: stem_words(text))
#df.head()

### Lemmatization

In [69]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) 
                     for word, pos in pos_tagged_text])
    #return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

#ex = "hello I am buying a fruit"
#lemmatize_words(ex)
#df["raw_lemmatized"] = df["raw"].apply(lambda text: lemmatize_words(text))
#df.head()

### Tokenisation

In [79]:
import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

#df['tweet_tokenized']=df["tweet_lower"].apply(lambda text: tokenize(text))

# 3. Feature engineering

### Tweet length

In [ ]:
df['length']=df['raw'].apply(lambda x:len(x)-x.count(" "))

In [ ]:
plt.plot(df.length)

### Punctuation percentage

In [ ]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")),3)*100

df['punct']=df['raw'].apply(count_punct)

In [ ]:
plt.plot(df.punct)

# 4. Data analysis

In [ ]:
bins=np.linspace(0,110,40)
plt.hist(df[df.has_reply==True]['punct'],bins, alpha=0.5, normed=True, label='Urgent')
plt.hist(df[df.has_reply==False]['punct'],bins, alpha=0.5, normed=True, label='Non-Urgent')
plt.legend(loc='upper left')
plt.show()

Les features calculées n'ont pas l'air de différencier les tweets

In [ ]:
#Distribution of reply times
plt.plot(df['reply_time'])

# 5. Models

# 5A. Model creation 

## 1. Final pre-processing

In [77]:
df["clean_tweet"] = df["raw"].apply(lambda text: remove_uppercase(text))
df["clean_tweet"] = df["clean_tweet"].apply(lambda text: remove_punctuation(text))
df["clean_tweet"] = df["clean_tweet"].apply(lambda text: remove_stopwords(text))
#df["clean_tweet"] = df["clean_tweet"].apply(lambda text: remove_frequent_words(text))
df["clean_tweet"] = df["clean_tweet"].apply(lambda text: lemmatize_words(text))

In [80]:
df["clean_tweet_tokenized"] = df["clean_tweet"].apply(lambda text: tokenize(text))

In [81]:
df.columns

Index(['raw', 'reply_time', 'has_reply', 'clean_tweet',
       'clean_tweet_tokenized'],
      dtype='object')

### Create dataset

In [87]:
matrix = CountVectorizer()
X = matrix.fit_transform(df.clean_tweet).toarray()

In [88]:
# Choose value to predict
y = df.has_reply

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.80,random_state=44)

## 2. Vectorization

In [ ]:
def build_model(mode):
    # Intent to use default paramaters for show case
    vect = None
    if mode == 'count':
        vect = CountVectorizer()
    elif mode == 'tf':
        vect = TfidfVectorizer(use_idf=False, norm='l2')
    elif mode == 'tfidf':
        vect = TfidfVectorizer()
    #add n-gram
    else:
        raise ValueError('Mode should be either count or tfidf')
    
    return Pipeline([
        ('vect', vect),
        ('clf' , LogisticRegression(solver='newton-cg',n_jobs=-1))
    ])

## 3. Choose model

### Random Forest

In [93]:
rf = RandomForestClassifier()

In [94]:
rf.fit(X_train,y_train)

C:\Users\carol\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [95]:
y_pred = rf.predict(X_test)

In [98]:
classifier=rf

# 5B. Model evaluation

In [ ]:
print('Performance on test set :')
print('\n')

print('Accuracy : %.3f ' % accuracy_score(y_test, y_pred))


### Feature importance

### Confusion Matrix

In [96]:
## PRINT SCORES

accuracy = accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)
precision_score = precision_score(y_test, y_pred)
recall_score = recall_score(y_test, y_pred)


string = "RESULTATS \n \
Accuracy  : {accuracy}% \n \
F1 score  : {f1_score}\n \
Precision : {precision}% (how many selected items are relevant)\n \
Recall    : {recall}% (how many relevant items are selected)\n \
\n\n\
STATISTICS\n\
 Proportion of positives in test set    : {positives}%\n\
 Proportion of positives in predictions : {predicted_positives}%\
"

print(string.format(accuracy = round(100*accuracy,2),
                    f1_score = round(f1_score,4), 
                    precision = round(100*precision_score,2), 
                    recall = round(100*recall_score,2),
                    positives = round(100*y_test.sum()/len(y_test),2),
                    predicted_positives = round(100*y_pred.sum()/len(y_pred),2),
                   ))

RESULTATS 
 Accuracy  : 75.44% 
 F1 score  : 0.85
 Precision : 78.87% (how many selected items are relevant)
 Recall    : 92.16% (how many relevant items are selected)
 

STATISTICS
 Proportion of positives in test set    : 75.51%
 Proportion of positives in predictions : 88.23%


In [99]:
## DISPLAY ROC CURVE
roc_curve = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

plt.plot(roc_curve[0], roc_curve[1], color='darkorange',
         lw=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

No handles with labels found to put in legend.
